In [1]:
import pandas as pd

df = pd.read_csv("customer_shopping_behavior.csv")

In [2]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [4]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [5]:
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [6]:
df.isnull().sum()

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

In [7]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})

In [8]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [9]:
df['age_group'] = pd.qcut(df['age'], q=4, labels = ['Young Adult','Adult','Middle Aged','Senior'])

In [10]:
# purchase frequency days

frequency_mapping = {
    'Fortnightly': 14,
    'Weekly': 7,
    'Monthly': 30,
    'Quarterly': 90,
    'Bi-Weekly': 14,
    'Annually': 365,
    'Every 3 Months': 90
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)

In [11]:
df[['purchase_frequency_days','frequency_of_purchases']].head()

,purchase_frequency_days,frequency_of_purchases
0,14,Fortnightly
1,14,Fortnightly
2,7,Weekly
3,7,Weekly
4,365,Annually


In [12]:
pip install pymysql sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [15]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

username = "root"
password = quote_plus("Redhawk@5")   # 🔥 THIS LINE FIXES EVERYTHING
host = "localhost"
port = "3306"
database = "customer_behavior"

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}",
    echo=True)
df.to_sql("customer",
    engine,
    if_exists="replace",
    index=False)

2025-12-19 20:04:03,040 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-12-19 20:04:03,041 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-19 20:04:03,045 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-12-19 20:04:03,047 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-19 20:04:03,050 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-12-19 20:04:03,052 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-19 20:04:03,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-19 20:04:03,067 INFO sqlalchemy.engine.Engine DESCRIBE `customer_behavior`.`customer`
2025-12-19 20:04:03,070 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-19 20:04:03,080 INFO sqlalchemy.engine.Engine DESCRIBE `customer_behavior`.`customer`
2025-12-19 20:04:03,082 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-19 20:04:03,091 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `customer_behavior`
2025-12-19 20:04:03,092 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-19 20

3900